### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()



+--------+---+----------+------+
|    Name|age|Experience|Salary|
+--------+---+----------+------+
|Sandipan| 31|        10| 30000|
|  Sumana| 30|         8| 25000|
| Krishna| 29|         4| 20000|
|    Paul| 24|         3| 20000|
|  Harsha| 21|         1| 15000|
| Shubham| 23|         2| 18000|
+--------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [7]:
# [Age,Experience]----> new feature--->independent feature

In [8]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [9]:
output=featureassembler.transform(training)

In [10]:
output.show()

+--------+---+----------+------+--------------------+
|    Name|age|Experience|Salary|Independent Features|
+--------+---+----------+------+--------------------+
|Sandipan| 31|        10| 30000|         [31.0,10.0]|
|  Sumana| 30|         8| 25000|          [30.0,8.0]|
| Krishna| 29|         4| 20000|          [29.0,4.0]|
|    Paul| 24|         3| 20000|          [24.0,3.0]|
|  Harsha| 21|         1| 15000|          [21.0,1.0]|
| Shubham| 23|         2| 18000|          [23.0,2.0]|
+--------+---+----------+------+--------------------+



In [11]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data=output.select("Independent Features","Salary")

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [14]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [15]:
### Coefficients
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [16]:
### Intercepts
regressor.intercept

16079.136690647425

In [17]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [18]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17214.09079632846|
+--------------------+------+-----------------+



In [19]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(785.909203671541, 617653.2764156357)